In [42]:
from pprint import pprint
import urllib
from bs4 import BeautifulSoup
from webdriver_manager.firefox import GeckoDriverManager
import time
import sys
import datetime

from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

In [85]:
# Given a URL string, opens the URL in a headless Firefox instance and returns a reference to the webdriver object

def obtain_driver(url):
    buttons = []
    driver_options = Options()
#     driver_options.headless = True
    
    print("\n\n*****" + str(datetime.datetime.now()) + "*****")
    
    driver = webdriver.Firefox(options=driver_options, executable_path=GeckoDriverManager().install())
    
    # WARNING, THIS IS SPECIFIC TO THIS MACHINE (retrieves the installed uBlock Origin from the Firefox on this computer.) Need a computer-agnostic way.
    driver.install_addon("~/.mozilla/firefox/f4t6w0s5.default-release/extensions/uBlock0@raymondhill.net.xpi", temporary=True)

    driver.get(url)
    
    time.sleep(.5)
    
    return driver

In [43]:
# Clicks on element and waits for a page load. driver == webdriver, element == string (class or css_selector), find_elements_by == integer
# find_elements_by: 0 for css_selector, 1 for class_name

def click_and_wait(driver, element, find_elements_by):
    old_driver = driver.find_element_by_tag_name('html')
    
    WebDriverWait(driver, 10).until(EC.staleness_of(old_driver))
    
    try:
        if find_elements_by == 0:
            button = driver.find_element_by_css_selector(element)
        else:
            button = driver.find_element_by_class_name(element)
            
        driver.execute_script("arguments[0].click()", button)
    except:
        print("No button")

In [93]:
# Given a DOI, uses obtain_driver to open up the search site and search the DOI to open up the paper in the page. Returns the webdriver associated with it.

def search_paper(doi, driver, doi_urls):
    if "www.researchgate.net/search" in driver.current_url:
        search_input = "search-form__input"
        search_button = "search-form__left-button"
    else:
        search_input = "lite-page__header-search-input"
        search_button = "lite-page__header-search-button"
    
    print("\n---" + "New search " + str(datetime.datetime.now()) + "---")
    print("DOI: " + doi)y
    
    
    search_field_list = driver.find_element_by_class_name(search_input)
    time.sleep(.25) # added for stability
    search_field = search_field_list
    time.sleep(.25)
    search_field.clear()
    time.sleep(.25)
    search_field.send_keys(doi)
    
    enter_field = driver.find_element_by_class_name(search_button)
    driver.execute_script("arguments[0].click()", enter_field)
  
    click_and_wait(driver, "a[href*='search/data']", 0)

    time.sleep(.5)
    
    if "www.researchgate.net/search" in driver.current_url:
        print("Paper not found (search)")
        
        return None
    else: 
        print("Success")
        doi_urls[doi] = driver.current_url
        
        return driver

In [69]:
# Testing click_and_wait

driver = obtain_driver("https://www.researchgate.net/search")
# time.sleep(1)

search_field_list = driver.find_element_by_class_name("search-form__input")
time.sleep(.5) # added for stability
search_field = search_field_list
time.sleep(.5)
search_field.clear()
time.sleep(.5)
search_field.send_keys("10.1021/acs.iecr.5b03509")

enter_field = driver.find_element_by_class_name("search-form__left-button")
driver.execute_script("arguments[0].click()", enter_field)
# time.sleep(.5)
# click_and_wait(driver, "search-form__left-button", 0)


click_and_wait(driver, "a[href*='search/data']", 0)
# data_field = driver.find_elements_by_css_selector("a[href*='search/data']")
# time.sleep(.5) # added for stability
# print(data_field)
# driver.execute_script("arguments[0].click()", data_field[0])

click_and_wait(driver, "show-more-authors", 1)

# WebDriverWait(driver, 10).until(
#     EC.presence_of_all_elements_located((By.CLASS_NAME, "nova-e-text nova-e-text--size-m nova-e-text--family-sans-serif nova-e-text--spacing-none nova-e-text--color-inherit nova-e-text--clamp nova-v-person-list-item__title"))
# )

time.sleep(.5)

# print(driver.find_element_by_xpath("/html/body/div[2]/main/section/section[1]/ul/ul"))

soup = BeautifulSoup(driver.page_source, 'html.parser')
thing = soup.find_all('div', {"class": "nova-e-text nova-e-text--size-m nova-e-text--family-sans-serif nova-e-text--spacing-none nova-e-text--color-inherit nova-e-text--clamp nova-v-person-list-item__title"})
for stuff in thing:
    author_url = stuff.find('a').get("href")
    print(author_url)
#     if "Ribeiro" in author_url:
#         print(str(author_url) + "<-------------")


# search_field_list = driver.find_elements_by_class_name("lite-page__header-search-input")
# time.sleep(.5) # added for stability
# search_field = search_field_list[0]
# time.sleep(.5)
# search_field.clear()
# time.sleep(.5)

# search_field.send_keys("10.1021/acs.iecr.5b03509")
# enter_field = driver.find_elements_by_class_name("lite-page__header-search-button")[0]
# driver.execute_script("arguments[0].click()", enter_field)



*****2019-06-24 10:59:09.035501*****

Checking for linux64 geckodriver:v0.24.0 in cache
Driver found in /home/local/NIST/jfl2/.wdm/geckodriver/v0.24.0/linux64/geckodriver
https://www.researchgate.net/profile/Juliana_Coelho5
https://www.researchgate.net/profile/Ana_Ribeiro25
https://www.researchgate.net/profile/Alexandre_Ferreira4
https://www.researchgate.net/scientific-contributions/2082644774_Sebastiao_MP_Lucena
https://www.researchgate.net/profile/Alirio_Rodrigues
https://www.researchgate.net/profile/Diana_Azevedo


In [4]:
with open("./stored_authors/authors_and_papers.txt", encoding="utf8") as papers_file:
    authors_and_papers = eval(papers_file.read())

In [5]:
doi_urls = {}

for paper_list in authors_and_papers.values():
    for paper in paper_list:
        doi_urls[paper] = None

In [6]:
original_file_descriptor = sys.stdout

In [7]:
sys.stdout = open("./paper_log.txt", "a")

In [94]:
overall_first_time = time.time()
blah = {}

driver = obtain_driver("https://www.researchgate.net/search")
time.sleep(1)

doi_list = ['10.1016/j.carbon.2009.06.046', '10.1016/j.coal.2004.05.002', '10.1016/j.coal.2005.07.003', '10.1016/j.coal.2007.01.005', '10.1016/j.coal.2010.08.013', '10.1016/s0166-5162(02)00078-2', '10.1016/s0375-6742(03)00122-5']

for doi in doi_list:
    search_paper(doi, driver, blah)
    time.sleep(.5)

# for paper in doi_urls:
#     try:
#         search_paper(paper, driver, doi_urls)
#     except Exception as e:
#         print("********* ERROR: " + str(e) + "*********")
        
#         driver.quit()
        
#         driver = obtain_driver("https://www.researchgate.net/search")
#         time.sleep(1) # maybe bring this up to since it can create a loop of Not Found -> Error (loads paper's page too late, then the site tries to enter/click stuff that doesn't exist (search_paper))
#     sys.stdout.flush()
#     time.sleep(.8)

driver.quit()

final_time = time.time() - overall_first_time
print("\n" + str(final_time))



*****2019-06-24 11:11:03.724490*****

Checking for linux64 geckodriver:v0.24.0 in cache
Driver found in /home/local/NIST/jfl2/.wdm/geckodriver/v0.24.0/linux64/geckodriver
https://www.researchgate.net/search

---New search 2019-06-24 11:11:14.489480---
DOI: 10.1016/j.carbon.2009.06.046
Success
https://www.researchgate.net/publication/222517880_European_inter-laboratory_comparison_of_high_pressure_CO2_sorption_isotherms_I_Activated_carbon

---New search 2019-06-24 11:11:17.206835---
DOI: 10.1016/j.coal.2004.05.002
Success
https://www.researchgate.net/publication/223844942_Methane_and_carbon_dioxide_adsorption-diffusion_experiments_on_coal_Upscaling_and_modeling

---New search 2019-06-24 11:11:19.726545---
DOI: 10.1016/j.coal.2005.07.003
Success
https://www.researchgate.net/publication/228825862_Investigation_of_high-pressure_selective_adsorptiondesorption_behaviour_of_CO2_and_CH4_on_coals_An_experimental_study

---New search 2019-06-24 11:11:22.272608---
DOI: 10.1016/j.coal.2007.01.005

In [ ]:
sys.stdout = original_file_descriptor

In [9]:
with open("./stored_authors/doi_url_pairings.txt", "w") as dup_file:
    pprint(doi_urls, stream = dup_file)

In [12]:
print(doi_urls['10.1016/j.carbon.2009.06.046'])

https://www.researchgate.net/publication/222517880_European_inter-laboratory_comparison_of_high_pressure_CO2_sorption_isotherms_I_Activated_carbon


In [22]:
for paper in doi_urls:
    print(paper)

In [34]:
print(len(doi_urls))

1550
